In [3]:
library(data.table)
library(Rmisc)
library(ggplot2)
library(dplyr)
library(reshape2)
library(bit)
library(zoo)
library(Hmisc)
library(boot)

## 1. Read raw data and merge them

In [8]:
DIR = "../data/"

rws=fread(paste0(DIR,"rws.all.csv"))

#-Named-entity recognition for further specifications on racially unidentified users
ls.hn.ner=fread(paste0(DIR,"df_listings_host_name_extended.csv"))
rws=merge(rws,ls.hn.ner, by="listing_id",type="left")

guest=fread(paste0(DIR,"guest.attribute.csv"))
colnames(guest)[c(8)]=c("g.ethnicity")
guest=guest[,c("guest_id","g.ethnicity")]
rws.gh=left_join(rws,guest, by="guest_id")

#-Note that h.num_face==0 represents hots without any human face in their profile photos.
#-In this case, h.ethnicity is empty string.
#-Additional filtering is required depending on analysis,
#-since profile photos with more than 1 human face also have empty string in the h.ethnicity column.
host=fread(paste0(DIR,"host.attribute.csv"))
colnames(host)[c(5,8)]=c("h.num_face","h.ethnicity")
host=host[,c("listing_id","host_id","h.num_face","h.ethnicity")]
rws.gh=left_join(rws.gh,host, by=c("listing_id","host_id"))

tmp_host=host
tmp_host$h.ethnicity=ifelse(tmp_host$h.num_face==0, "RA", tmp_host$h.ethnicity)

#-Remove those racially unidentified guests
rws.gh=rws.gh[rws.gh$g.ethnicity!="",]

#-Label RA for racially ambiguous hosts
rws.gh$h.ethnicity=ifelse(rws.gh$h.num_face==0, "RA", rws.gh$h.ethnicity)

#-Remove those racially unidentified hosts (with multiple human faces in their profile photos)
rws.gh=rws.gh[rws.gh$h.ethnicity!="",]

In [9]:
ls.meta.info=fread(paste0(DIR,"ls.with.rws.csv"))
h.meta.info=fread(paste0(DIR,"host.attribute.csv"))
zip2neighborhood=fread(paste0(DIR,"nyc_zip2neighborhood.csv"))

In [10]:
zip2neighborhood$ZipCode=as.character(zip2neighborhood$ZipCode)

## 2. Process reputation and endorsement signals

In [11]:
df=rws.gh

df=df %>% group_by(listing_id) %>% arrange(date) %>%
  mutate(cum.cnt=row_number()-1,
         rws.race=lag(g.ethnicity, n = 1, default = NA))

#-Create column and add counter
df$g.W.cnt=ifelse(df$rws.race=="WHITE", 1, 0)
df$g.B.cnt=ifelse(df$rws.race=="BLACK", 1, 0)
df$g.A.cnt=ifelse(df$rws.race=="ASIAN", 1, 0)

#-Sort reviews for each listing by date
#-Then roll sum racial compositions in the most recent 6 reviews for each booking
#-Note that the most 6 reviews are the proxy of front-page reviews
df = df %>% group_by(listing_id) %>% arrange(date) %>%
    mutate(top6.W=rollsumr(g.W.cnt==1, 6, fill=NA),
           top6.B=rollsumr(g.B.cnt==1, 6, fill=NA),
           top6.A=rollsumr(g.A.cnt==1, 6, fill=NA))

#-Code the first review's the most recent previous reviewer's race as 0
#-and then count all the previous reviewers' races cumulatively
df$rws.race[which(is.na(df$rws.race))]=0

df = df %>% group_by(listing_id) %>% arrange(date) %>% 
    mutate(g.W.cnt=cumsum(rws.race=="WHITE"),
           g.B.cnt=cumsum(rws.race=="BLACK"),
           g.A.cnt=cumsum(rws.race=="ASIAN"))

## 3. Screen instant bookings and split data into different time periods

In [12]:
#-Extract only listings that allow instant booking
df=as.data.table(df)
df=df[which(df$instant_bookable=="t"),]

In [13]:
tmp_2018 = df[df$date >= as.Date("2018-01-01")]

In [14]:
tmp_2018$zipcode = ls.meta.info[match(tmp_2018$listing_id, ls.meta.info$listing_id), zipcode]
tmp_2018 = merge(tmp_2018, zip2neighborhood, by.x="zipcode", by.y="ZipCode")

# ----------------------------------------------------------------------------------------

In [15]:
create_ctf_1 = function(df.x) {
    df.tmp.ctf=df.x
    df.tmp.ctf$pairing.status=0
    df.tmp.ctf$g.ethnicity=ifelse(df.tmp.ctf$g.ethnicity=="BLACK" |
                                  df.tmp.ctf$g.ethnicity=="ASIAN",
                                  "WHITE", "BLACK")
    return(df.tmp.ctf)
}

create_ctf_2 = function(df.x) {
    df.tmp.ctf=df.x
    df.tmp.ctf$pairing.status=0
    df.tmp.ctf$g.ethnicity=ifelse(df.tmp.ctf$g.ethnicity=="BLACK" |
                                  df.tmp.ctf$g.ethnicity=="WHITE",
                                  "ASIAN", "BLACK")
    return(df.tmp.ctf)
}

create_ctf = function(df.x) {
    return(rbind(create_ctf_1(df.x),create_ctf_2(df.x)))
}

## Fig. S9A

### 1.1 Compute normed probabilities for guest-host pairing combinations

- Compute the weighted mean of means based on the neighborhood-specific normed probabilities and the size of observations!
- Fully computed normed probabilities are available (check ***1.2*** without running code below in ***1.1***)

In [16]:
tmp=tmp_2018[, .N, by=Neighborhood]

options(warn=-1)
start_time=Sys.time()
l_summary_obs=list()
l_summary_diff=list()
for(n in unique(tmp$Neighborhood)){
    df.tmp.obs=tmp_2018[tmp_2018$Neighborhood==n]
    
    df.tmp.obs$pairing.status=1
    df.tmp.ctf=create_ctf(df.tmp.obs)
    df.obs.ctf=rbind(df.tmp.obs,df.tmp.ctf)
    summary.pairing.prob.obs=summarySE(df.obs.ctf, measurevar="pairing.status",
                                       groupvars=c("h.ethnicity", "g.ethnicity"))
    summary.pairing.prob.obs$neighborhood=n
    l_summary_obs[[n]]=summary.pairing.prob.obs
    
    df.tmp.rnd=df.tmp.obs
    
    shuffle_time=1000
    l_diff=list()
    for(x in 1:shuffle_time){
        df.tmp=df.tmp.rnd
        df.tmp$g.ethnicity=df.tmp.rnd[sample(nrow(df.tmp.rnd), replace=FALSE),]$g.ethnicity

        df.tmp.ctf=create_ctf(df.tmp)
        df.tmp.rnd.ctf=rbind(df.tmp,df.tmp.ctf)

        df.tmp.pairing.prob.obs=summarySE(df.tmp.rnd.ctf, measurevar="pairing.status",
                                          groupvars=c("h.ethnicity", "g.ethnicity"))
        df.tmp.diff=df.tmp.pairing.prob.obs[,1:2]
        df.tmp.diff$prob.diff=summary.pairing.prob.obs$pairing.status - df.tmp.pairing.prob.obs$pairing.status
        l_diff[[paste("obs.rnd.diff", x, sep=".")]]=df.tmp.diff
    }

    df.pairing.prob.diff=do.call(rbind.data.frame, l_diff)

    summary.pairing.prob.diff=summarySE(df.pairing.prob.diff, measurevar="prob.diff",
                                        groupvars=c("h.ethnicity", "g.ethnicity"))
    summary.pairing.prob.diff$ci=summary.pairing.prob.diff$sd*1.96
    summary.pairing.prob.diff$neighborhood=n
    l_summary_diff[[n]]=summary.pairing.prob.diff
}

end_time=Sys.time()
end_time - start_time
options(warn=0)

Time difference of 5.210044 mins

In [17]:
tmp_obs=do.call(rbind.data.frame, l_summary_obs)
tmp_normed=do.call(rbind.data.frame, l_summary_diff)

### 1.2 Visualize normed probabilities by race of guest and host

In [24]:
filename_obs="supp_fig_s9a_pairing.prob.obs_2018_neighborhood-based.RDS"
filename_diff="supp_fig_s9a_pairing.prob.diff_2018_neighborhood-based.RDS"
filename_pdf="supp_fig_s9a.pdf"

In [140]:
#-Read
tmp_obs=readRDS(paste0(DIR,filename_obs))
tmp_normed=readRDS(paste0(DIR,filename_diff))

In [26]:
samplewmean <- function(data, d) {
  return(weighted.mean(x=data[d,1], w=data[d,2]))
}

l_boot_mean=list()
for (x in unique(tmp_obs$h.ethnicity)) {
    for (y in unique(tmp_obs$g.ethnicity)) {
        df.tmp.obs=tmp_obs[(tmp_obs$h.ethnicity==x)&
                           (tmp_obs$g.ethnicity==y),]
        l.weight=df.tmp.obs$N/sum(df.tmp.obs$N)
        df.tmp.normed=tmp_normed[(tmp_normed$h.ethnicity==x)&
                                 (tmp_normed$g.ethnicity==y),]
        l.normed.prob=df.tmp.normed$prob.diff

        boot_mean=boot(data=cbind(l.normed.prob, l.weight), 
                       statistic=samplewmean, 
                       R=1000)

        l_boot_mean[[paste("boot_mean", x, y, sep=".")]]=data.frame(matrix(c(boot_mean$t0,sd(boot_mean$t),x,y), 1))
    }
}

tmp=do.call(rbind.data.frame, l_boot_mean)
colnames(tmp)=c("prob.diff", "sd", "h.ethnicity", "g.ethnicity")
tmp$ci=as.numeric(tmp$sd)*1.96
tmp$prob.diff=as.numeric(tmp$prob.diff)

In [27]:
tmp$h.ethnicity=ifelse(tmp$h.ethnicity=="RA",
                       "Racially\nUnidentified", tmp$h.ethnicity)
re_from="\\b([[:alpha:]])([[:alpha:]]+)"
tmp$h.ethnicity=gsub(re_from, "\\U\\1\\L\\2", tmp$h.ethnicity, perl=TRUE)
tmp$h.ethnicity=factor(tmp$h.ethnicity,
                       levels=c("Racially\nUnidentified","White","Asian","Black"),
                       ordered=TRUE)
tmp$g.ethnicity=factor(tmp$g.ethnicity,
                       levels=c("WHITE","ASIAN","BLACK"),
                       ordered=TRUE)
summary.pairing.prob.diff=tmp

In [28]:
l_guest.race.label=c(
    "BLACK"="Black Guest",
    "WHITE"="White Guest",
    "ASIAN"="Asian Guest")

dodge=position_dodge(width=0.5)
pdf(paste0("../output/",filename_pdf), width=3.17, height=4)
g = ggplot(summary.pairing.prob.diff, aes(x=h.ethnicity, y=prob.diff)) + 
  xlab("Host Race") + 
  ylab("Pairing Probability Compared to Random") +
  geom_point(position=dodge, size=2.5) +
  geom_errorbar(aes(ymin=prob.diff-ci, ymax=prob.diff+ci),
                size=0.75, width=0.25, position=dodge) +
  facet_grid(~g.ethnicity, margins=FALSE, switch="y",
             labeller=as_labeller(l_guest.race.label)) + theme_classic() +
  scale_y_continuous(limits=c(-0.04,0.04), breaks=seq(-0.04,0.04,0.02),
                     labels=c(seq(-0.04,0.04,0.02))) +
  theme(panel.border=element_rect(fill=NA, size=0.3),
        panel.grid.major=element_blank(),
        axis.text.x=element_text(angle=90, hjust=1, vjust=0.4)) +
  geom_hline(yintercept=0, linetype="dashed", color="#D55E00") +
  scale_color_grey(start=0.5, end=0.5) + theme(text=element_text(size=10))
plot(g)
dev.off()

pdf 
  2

In [29]:
fig_a=g

## Fig. S9B

### 1.1 Compute normed probabilities for guest-host pairing combinations

- Compute the weighted mean of means based on the neighborhood-specific normed probabilities and the size of observations!
- Fully computed normed probabilities are available (check ***1.2*** without running code below in ***1.1***)

In [31]:
tmp=tmp_2018[, .N, by=Neighborhood]

options(warn=-1)
start_time=Sys.time()
l_summary_obs=list()
l_summary_diff=list()
l_summary_obs_binary=list()
l_summary_diff_binary=list()
for(n in unique(tmp$Neighborhood)){
    df.tmp.obs=tmp_2018[tmp_2018$Neighborhood==n]
    df.tmp.obs=df.tmp.obs[, c("h.ethnicity", "g.ethnicity",
                          "top6.A", "top6.B", "top6.W"), with=FALSE]
    
    df.tmp.obs$pairing.status=1
    #-Cap to 5 due to small N for Asian and Black guests on SRE 5 and 6
    df.tmp.obs$top6.A=ifelse(df.tmp.obs$top6.A>5, 5, df.tmp.obs$top6.A)
    df.tmp.obs$top6.B=ifelse(df.tmp.obs$top6.B>5, 5, df.tmp.obs$top6.B)
    df.tmp.obs$top6.W=ifelse(df.tmp.obs$top6.W>5, 5, df.tmp.obs$top6.W)
    
    df.tmp.ctf=create_ctf(df.tmp.obs)
    df.obs.ctf=rbind(df.tmp.obs,df.tmp.ctf)
    
    df.obs.ctf$sre[df.obs.ctf$g.ethnicity=="WHITE"]=df.obs.ctf$top6.W[df.obs.ctf$g.ethnicity=="WHITE"]
    df.obs.ctf$sre[df.obs.ctf$g.ethnicity=="ASIAN"]=df.obs.ctf$top6.A[df.obs.ctf$g.ethnicity=="ASIAN"]
    df.obs.ctf$sre[df.obs.ctf$g.ethnicity=="BLACK"]=df.obs.ctf$top6.B[df.obs.ctf$g.ethnicity=="BLACK"]
    df.obs.ctf=df.obs.ctf[complete.cases(df.obs.ctf), ]
    
    summary.pairing.prob.obs=summarySE(df.obs.ctf, measurevar="pairing.status",
                                       groupvars=c("h.ethnicity",
                                                   "g.ethnicity",
                                                   "sre"))
    summary.pairing.prob.obs$neighborhood=n
    l_summary_obs[[n]]=summary.pairing.prob.obs
    
    df.tmp.rnd=df.tmp.obs
    
    shuffle_time=1000
    l_diff=list()
    for(x in 1:shuffle_time){
        df.tmp=df.tmp.rnd
        df.tmp$g.ethnicity=df.tmp.rnd[sample(nrow(df.tmp.rnd), replace=FALSE),]$g.ethnicity

        df.tmp.ctf=create_ctf(df.tmp)
        df.tmp.rnd.ctf=rbind(df.tmp,df.tmp.ctf)

        df.tmp.rnd.ctf$sre[df.tmp.rnd.ctf$g.ethnicity=="WHITE"]=df.tmp.rnd.ctf$top6.W[df.tmp.rnd.ctf$g.ethnicity=="WHITE"]
        df.tmp.rnd.ctf$sre[df.tmp.rnd.ctf$g.ethnicity=="ASIAN"]=df.tmp.rnd.ctf$top6.A[df.tmp.rnd.ctf$g.ethnicity=="ASIAN"]
        df.tmp.rnd.ctf$sre[df.tmp.rnd.ctf$g.ethnicity=="BLACK"]=df.tmp.rnd.ctf$top6.B[df.tmp.rnd.ctf$g.ethnicity=="BLACK"]
        df.tmp.rnd.ctf=df.tmp.rnd.ctf[complete.cases(df.tmp.rnd.ctf), ]
        df.tmp.pairing.prob.obs=summarySE(df.tmp.rnd.ctf, measurevar="pairing.status",
                                          groupvars=c("h.ethnicity", "g.ethnicity", "sre"))
        
        df.tmp.diff=df.tmp.pairing.prob.obs[,1:3]
        df.tmp.diff$prob.diff=summary.pairing.prob.obs$pairing.status - df.tmp.pairing.prob.obs$pairing.status
        l_diff[[paste("obs.rnd.diff", x, sep=".")]]=df.tmp.diff
    }

    df.pairing.prob.diff=do.call(rbind.data.frame, l_diff)

    summary.pairing.prob.diff=summarySE(df.pairing.prob.diff, measurevar="prob.diff",
                                        groupvars=c("h.ethnicity", "g.ethnicity", "sre"))
    summary.pairing.prob.diff$ci=summary.pairing.prob.diff$sd*1.96
    summary.pairing.prob.diff$neighborhood=n
    l_summary_diff[[n]]=summary.pairing.prob.diff
    
    #-For Fig. 3
    tmp.binary.obs=df.obs.ctf[df.obs.ctf$h.ethnicity!="RA",]
    tmp.binary.obs$h.ethnicity=ifelse(tmp.binary.obs$h.ethnicity==tmp.binary.obs$g.ethnicity,
                                      "Same-Race", "Other-Race")
    summary.pairing.prob.diff=summarySE(tmp.binary.obs, measurevar="pairing.status",
                                        groupvars=c("h.ethnicity", "g.ethnicity", "sre"))
    summary.pairing.prob.diff$ci=summary.pairing.prob.diff$sd*1.96
    summary.pairing.prob.diff$neighborhood=n
    l_summary_obs_binary[[n]]=summary.pairing.prob.diff
    
    tmp.binary.diff=df.pairing.prob.diff[df.pairing.prob.diff$h.ethnicity!="RA",]
    tmp.binary.diff$h.ethnicity=ifelse(tmp.binary.diff$h.ethnicity==tmp.binary.diff$g.ethnicity,
                                       "Same-Race", "Other-Race")
    summary.pairing.prob.diff=summarySE(tmp.binary.diff, measurevar="prob.diff",
                                        groupvars=c("h.ethnicity", "g.ethnicity", "sre"))
    summary.pairing.prob.diff$ci=summary.pairing.prob.diff$sd*1.96
    summary.pairing.prob.diff$neighborhood=n
    l_summary_diff_binary[[n]]=summary.pairing.prob.diff
}

end_time=Sys.time()
end_time - start_time
options(warn=0)

Time difference of 7.003013 mins

In [32]:
tmp_obs=do.call(rbind.data.frame, l_summary_obs)
tmp_normed=do.call(rbind.data.frame, l_summary_diff)

### 1.2 Visualize normed probabilities by race of guest and host

In [33]:
filename_obs="supp_fig_s9b_pairing.prob.obs_2018_neighborhood-based.RDS"
filename_diff="supp_fig_s9b_pairing.prob.diff_2018_neighborhood-based.RDS"
filename_pdf="supp_fig_s9b.pdf"

In [146]:
#-Read
tmp_obs=readRDS(paste0(DIR,filename_obs))
tmp_normed=readRDS(paste0(DIR,filename_diff))

In [35]:
samplewmean <- function(data, d) {
  return(weighted.mean(x=data[d,1], w=data[d,2]))
}

start_time=Sys.time()

l_boot_mean=list()
for (x in unique(tmp_obs$h.ethnicity)) {
    for (y in unique(tmp_obs$g.ethnicity)) {
        for (z in unique(tmp_obs$sre)) {
            df.tmp.obs=tmp_obs[((tmp_obs$h.ethnicity==x)&
                               (tmp_obs$g.ethnicity==y)&
                               (tmp_obs$sre==z)),]
            l.weight=df.tmp.obs$N/sum(tmp_obs$N)
            df.tmp.normed=tmp_normed[((tmp_normed$h.ethnicity==x)&
                                      (tmp_normed$g.ethnicity==y)&
                                      (tmp_normed$sre==z)),]
            l.normed.prob=df.tmp.normed$prob.diff

            boot_mean=boot(data=cbind(l.normed.prob, l.weight), 
                           statistic=samplewmean, 
                           R=1000)

            l_boot_mean[[paste("boot_mean", x, y, z, sep=".")]]=data.frame(matrix(c(boot_mean$t0,sd(boot_mean$t),x,y,z), 1))
        }    
    }
}

end_time=Sys.time()
end_time - start_time

tmp_boot=do.call(rbind.data.frame, l_boot_mean)
colnames(tmp_boot)=c("prob.diff", "sd", "h.ethnicity", "g.ethnicity", "sre")
tmp_boot$ci=as.numeric(tmp_boot$sd)*1.96
tmp_boot$prob.diff=as.numeric(tmp_boot$prob.diff)

Time difference of 1.012948 secs

In [36]:
summary.pairing.prob.diff=tmp_boot

summary.pairing.prob.diff$h.ethnicity=ifelse(summary.pairing.prob.diff$h.ethnicity=="RA",
                                             "Racially Unidentified", summary.pairing.prob.diff$h.ethnicity)

re_from="\\b([[:alpha:]])([[:alpha:]]+)"
summary.pairing.prob.diff$h.ethnicity=gsub(re_from, "\\U\\1\\L\\2", summary.pairing.prob.diff$h.ethnicity, perl=TRUE)
summary.pairing.prob.diff$h.ethnicity=ifelse(summary.pairing.prob.diff$h.ethnicity=="RA",
                                             "Racially Unidentified", summary.pairing.prob.diff$h.ethnicity)

summary.pairing.prob.diff$h.ethnicity=factor(summary.pairing.prob.diff$h.ethnicity,
                                             levels=c("Racially Unidentified","White","Asian","Black"),
                                             ordered=TRUE)

summary.pairing.prob.diff$g.ethnicity=factor(summary.pairing.prob.diff$g.ethnicity,
                                             levels=c("WHITE","ASIAN","BLACK"),
                                             ordered=TRUE)

summary.pairing.prob.diff$sre=factor(summary.pairing.prob.diff$sre)

In [37]:
tmp = summary.pairing.prob.diff[summary.pairing.prob.diff$h.ethnicity=="Racially Unidentified",]

l_guest.race.label=c(
    "BLACK"="Black Guest",
    "WHITE"="White Guest",
    "ASIAN"="Asian Guest")
cc=c("#899da4","#f1aa00","#be5d05","#2c2d4a")

dodge=position_dodge(width=0.5)
pdf(paste0("../output/",filename_pdf), width=3.17, height=4)
g = ggplot(tmp, aes(x=sre, y=prob.diff)) + 
  geom_line(position=dodge, size=0.75, aes(color=h.ethnicity, group=h.ethnicity)) +
  xlab("Number of Same-Race Endorsements\n(Out of 5 or More Front-Page Reviews)") + 
  # ylab("Pairing Probability Compared to Random") +
  ylab("") +
  geom_point(position=dodge, aes(color=h.ethnicity, group=h.ethnicity), size=2.5) +
  geom_errorbar(aes(ymin=prob.diff-ci, ymax=prob.diff+ci, color=h.ethnicity),
                size=0.75, width=0.15, position=dodge) +
  scale_color_manual(values=cc) +
  scale_x_discrete(labels=c("0","1","2","3","4","5+")) +
  facet_grid(g.ethnicity~., margins=FALSE, switch="y", scales="free_y",
             labeller=as_labeller(l_guest.race.label)) + labs(color="Host Race") +
             theme_classic() + theme(panel.border=element_rect(fill=NA, size=0.3)) +
  # coord_cartesian(ylim=c(-0.3,0.35)) +
  geom_hline(yintercept=0, linetype="dashed", color="#D55E00") +
  theme(legend.position="none",
        panel.grid.major=element_blank(),
        text=element_text(size=10))
plot(g)
dev.off()

pdf 
  2

In [38]:
fig_b=g

## Fig. S9C

### 1.1 Compute normed probabilities for guest-host pairing combinations

- Normed probabilities are computed in the previous section (with **Fig. S9B**).

In [39]:
tmp_obs=do.call(rbind.data.frame, l_summary_obs_binary)
tmp_normed=do.call(rbind.data.frame, l_summary_diff_binary)

### 1.2 Visualize normed probabilities by race of guest and host

In [40]:
filename_obs="supp_fig_s9c_pairing.prob.obs_2018_neighborhood-based.RDS"
filename_diff="supp_fig_s9c_pairing.prob.diff_2018_neighborhood-based.RDS"
filename_pdf="supp_fig_s9c.pdf"

In [152]:
#-Read
tmp_obs=readRDS(paste0(DIR,filename_obs))
tmp_normed=readRDS(paste0(DIR,filename_diff))

In [42]:
l_boot_mean=list()
for (x in unique(tmp_obs$h.ethnicity)) {
    for (y in unique(tmp_obs$g.ethnicity)) {
        for (z in unique(tmp_obs$sre)) {
            df.tmp.obs=tmp_obs[((tmp_obs$h.ethnicity==x)&
                               (tmp_obs$g.ethnicity==y)&
                               (tmp_obs$sre==z)),]
            l.weight=df.tmp.obs$N/sum(tmp_obs$N)
            df.tmp.normed=tmp_normed[((tmp_normed$h.ethnicity==x)&
                                      (tmp_normed$g.ethnicity==y)&
                                      (tmp_normed$sre==z)),]
            l.normed.prob=df.tmp.normed$prob.diff

            boot_mean=boot(data=cbind(l.normed.prob, l.weight), 
                           statistic=samplewmean, 
                           R=1000)

            l_boot_mean[[paste("boot_mean", x, y, z, sep=".")]]=data.frame(matrix(c(boot_mean$t0,sd(boot_mean$t),x,y,z), 1))
        }    
    }
}

tmp_boot=do.call(rbind.data.frame, l_boot_mean)
colnames(tmp_boot)=c("prob.diff", "sd", "h.ethnicity", "g.ethnicity", "sre")
tmp_boot$ci=as.numeric(tmp_boot$sd)*1.96
tmp_boot$prob.diff=as.numeric(tmp_boot$prob.diff)

summary.pairing.prob.diff=tmp_boot
summary.pairing.prob.diff$h.ethnicity=factor(summary.pairing.prob.diff$h.ethnicity,
                                             levels=c("Same-Race","Other-Race"),
                                             ordered=TRUE)
summary.pairing.prob.diff$g.ethnicity=factor(summary.pairing.prob.diff$g.ethnicity,
                                             levels=c("WHITE","ASIAN","BLACK"),
                                             ordered=TRUE)
summary.pairing.prob.diff$sre=factor(summary.pairing.prob.diff$sre)

In [44]:
l_guest.race.label=c(
    "BLACK"="Black Guest",
    "WHITE"="White Guest",
    "ASIAN"="Asian Guest")

cc=c("#be5d05","#2c2d4a")

dodge=position_dodge(width=0.5)
pdf(paste0("../output/",filename_pdf), width=3.17, height=4)
g = ggplot(summary.pairing.prob.diff, aes(x=sre, y=prob.diff)) + 
  geom_line(position=dodge, size=0.75, aes(color=h.ethnicity, group=h.ethnicity)) +
  xlab("Number of Same-Race Endorsements\n(Out of 5 or More Front-Page Reviews)") + 
  # ylab("Pairing Probability Compared to Random") +
  ylab("") +
  geom_point(position=dodge, aes(color=h.ethnicity, group=h.ethnicity), size=2.5) +
  geom_errorbar(aes(ymin=prob.diff-ci, ymax=prob.diff+ci, color=h.ethnicity),
                size=0.9, width=0.25, position=dodge) +
  scale_color_manual(values=cc) +
  scale_x_discrete(labels=c("0","1","2","3","4","5+")) +
  facet_grid(g.ethnicity~., margins=FALSE, switch="y", scales="free_y",
             labeller=as_labeller(l_guest.race.label)) + labs(color="Host Race") +
             theme_classic() + theme(panel.border=element_rect(fill=NA, size=0.3)) +
  # coord_cartesian(ylim=c(-0.3,0.4)) +
  geom_hline(yintercept=0, linetype="dashed", color="#D55E00") +
  theme(legend.direction="vertical",
        legend.position=c(0.6,0.77),
        legend.justification="left",
        legend.margin=margin(0, unit="cm"),
        legend.spacing.y=unit(0.1,"cm"),
        legend.key.size=unit(0.85,"lines"),
        panel.grid.major=element_blank(),
        text=element_text(size=10))
plot(g)
dev.off()

pdf 
  2

In [45]:
fig_c=g

# ----------------------------------------------------------------------------------------

In [46]:
library(cowplot)

g <- plot_grid(fig_a,fig_b,fig_c,
               labels="AUTO", label_size=12,
               ncol=3)

In [49]:
pdf(file="../output/supp_fig_s9.pdf", width=9.5, height=4)
print(g)
dev.off()

pdf 
  2